# Backend things

This needs to be tridied up, tested and turned into a thing. Ignore first cell for now - usage is in the next cell.

### What does it do?

finds sparce measures and outputs a json file telling you what and where the problem is.

In [4]:

import json
from os import listdir
from os.path import isfile, join

import requests
from pathlib import Path
import pandas as pd

COMMON_REF = "https://raw.githubusercontent.com/GSS-Cogs/ref_common/master/columns.csv"


class Column(object):
    
    # TODO - no chance this is the right way to do it
    def __init__(self,arg_list):
        self.title = arg_list[0]
        self.name = arg_list[1]
        self.component_attachment = arg_list[2]
        self.property_template = arg_list[3]
        self.value_template = arg_list[4]
        self.datatype = arg_list[5]
        self.value_transformation = arg_list[6]
        self.regex = arg_list[7]
        self.range = arg_list[8]

def get_column_lookup(raw_column_ref):
    
    column_defs = []
    df_common = pd.read_csv(COMMON_REF)
    df_family = pd.read_csv(raw_column_ref)
    
    for df in [df_common, df_family]:
        for i, row in df.iterrows():
            column_defs.append(Column([row[x] for x in df.columns.values]))
    
    return column_defs
    

def check_sparsity(raw_column_ref, base_path=None, http=None):

    if http is None:
        schemas = [f for f in listdir(Path(base_path)) if f.endswith("schema.json")]
    else:
        schemas = [http]
    print("All schemas found:")
    print("\n".join(schemas))
    
    columns = get_column_lookup(raw_column_ref)
    dimensions = [x for x in columns if x.component_attachment == "qb:dimension"]
    print("\nColumn references taken from:")
    print(COMMON_REF)
    print(raw_column_ref)
    print()
    
    for schema in schemas:
        print("For schema: ", schema)
        
        if http == None:
            schema_path = Path(schema) if base_path is None else Path(Path(base_path) / Path(schema))
            with open(schema_path, "r") as f:
                schema_as_dict = json.load(f)
        else:
            schema_as_dict = requests.get(http).json()
             
        obs_table = [x for x in schema_as_dict["tables"] if isinstance(x["tableSchema"], dict)][0]
        
        if http == None:
            obs_table_path = Path(obs_table["url"]) if base_path is None else Path(Path(base_path) / Path(obs_table["url"]))
        else:
            chop = len(http.split("/")[-1])
            obs_table_path = http[:-chop] + obs_table["url"]
        df = pd.read_csv(obs_table_path)
        
        # Get a lookup of name:title from the tableSchema:columns list
        column_lookup = { x["titles"]: x["name"] for x in  obs_table["tableSchema"]["columns"] }
        
        dimension_keys = [c.name for c in dimensions]
        obs_file_keys = obs_table["tableSchema"]["primaryKey"]
        cols_we_care_about = [x for x in obs_file_keys if x in dimension_keys]

        # Should be 1 line -after- we add a Growth reference
        for col in df.columns.values:
            if col != "Growth":
                if column_lookup[col] not in cols_we_care_about:
                    df = df.drop(col, axis=1)
                    
        # Mash each line (other than measure type) into one cell of a column then look for duplicates
        # i.e - number of duplicates of EVERY line should == measures.
        
        df["compositeKey"] = "" 
        for col in [x for x in df.columns.values if x != "Measure Type" and x != "compositeKey"]:
            df["compositeKey"] = df["compositeKey"]+df[col]
            
        # Compare measure we should have with measures we do have
        measures = df["Measure Type"].unique().tolist()
        failure_dict = {}
        for item in df["compositeKey"].unique().tolist():
            df_slice = df[df["compositeKey"] == item]
            if len(df_slice) != len(measures):
                missing = [x for x in measures if x not in df_slice["Measure Type"].unique().tolist()]
                if len(missing) > 0:
                    failure_dict[item] = {"missing": missing, "have": [x for x in measures if x not in missing]}
            
        if failure_dict != {}:
            print("Sparse measures detected! outputting failure information to....")
            output = "measure-fails-{}.json".format(obs_table["url"])
            with open(output, "w") as f:
                json.dump(failure_dict, f)
            print(output, "\n")
        else:
            print("No measure sparsity detected.")
    

# Usage

To run against a schema on jenskins do: `check_sparsity(<ref>, http=<schema url>)`

To run against a local folder full of schemas (from that folder) do: `check_sparsity(<ref>)`

To run against a local folder full of schemas from a parent folder (i.e to check contents of /out) do:`check_sparsity(<ref>, base_path=<relative path>)`

### some examples:

`check_sparsity("https://raw.githubusercontent.com/GSS-Cogs/family-trade/master/reference/columns.csv", base_path="out")`


`check_sparsity("https://raw.githubusercontent.com/GSS-Cogs/family-trade/master/reference/columns.csv", 
               http="https://ci.floop.org.uk/job/GSS_data/job/Trade/job/ONS-International-exports-of-services-from-subnational-areas-of-the-UK/lastSuccessfulBuild/artifact/datasets/ONS-International-exports-of-services-from-subnational-areas-of-the-UK/out/observations.csv-schema.json")`


In [5]:

# check a cube with no measure sparsity
check_sparsity("https://raw.githubusercontent.com/GSS-Cogs/family-trade/master/reference/columns.csv",
               http="https://ci.floop.org.uk/job/GSS_data/job/Trade/job/ONS-International-exports-of-services-from-subnational-areas-of-the-UK/lastSuccessfulBuild/artifact/datasets/ONS-International-exports-of-services-from-subnational-areas-of-the-UK/out/observations.csv-schema.json")

All schemas found:
https://ci.floop.org.uk/job/GSS_data/job/Trade/job/ONS-International-exports-of-services-from-subnational-areas-of-the-UK/lastSuccessfulBuild/artifact/datasets/ONS-International-exports-of-services-from-subnational-areas-of-the-UK/out/observations.csv-schema.json

Column references taken from:
https://raw.githubusercontent.com/GSS-Cogs/ref_common/master/columns.csv
https://raw.githubusercontent.com/GSS-Cogs/family-trade/master/reference/columns.csv

For schema:  https://ci.floop.org.uk/job/GSS_data/job/Trade/job/ONS-International-exports-of-services-from-subnational-areas-of-the-UK/lastSuccessfulBuild/artifact/datasets/ONS-International-exports-of-services-from-subnational-areas-of-the-UK/out/observations.csv-schema.json
No measure sparsity detected.


In [6]:

# check a cube that has measure sparsity
check_sparsity("https://raw.githubusercontent.com/GSS-Cogs/family-trade/master/reference/columns.csv",
               http="https://ci.floop.org.uk/job/GSS_data/job/Trade/job/ONS-Quarterly-National-Accounts/42/artifact/datasets/ONS-Quarterly-National-Accounts/out/quarterly-national-accounts-gdp-data-tables-gdp-per-head.csv-schema.json")

All schemas found:
https://ci.floop.org.uk/job/GSS_data/job/Trade/job/ONS-Quarterly-National-Accounts/42/artifact/datasets/ONS-Quarterly-National-Accounts/out/quarterly-national-accounts-gdp-data-tables-gdp-per-head.csv-schema.json

Column references taken from:
https://raw.githubusercontent.com/GSS-Cogs/ref_common/master/columns.csv
https://raw.githubusercontent.com/GSS-Cogs/family-trade/master/reference/columns.csv

For schema:  https://ci.floop.org.uk/job/GSS_data/job/Trade/job/ONS-Quarterly-National-Accounts/42/artifact/datasets/ONS-Quarterly-National-Accounts/out/quarterly-national-accounts-gdp-data-tables-gdp-per-head.csv-schema.json
Sparse measures detected! outputting failure information to....
measure-fails-quarterly-national-accounts-gdp-data-tables-gdp-per-head.csv.json 

